In [1]:
import os
import phoenix as px

# 將 PHOENIX_COLLECTOR_ENDPOINT 指向本機實際 IP
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://192.168.1.122:6006"

client = px.Client()
dataset = client.get_dataset(
    name="my-chinese-ods-dataset",
    version_id="RGF0YXNldFZlcnNpb246OQ=="
)

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


In [2]:
import phoenix
print(phoenix.__version__)

9.5.0


In [7]:
print(dataset)

Dataset(id='RGF0YXNldDo3', version_id='RGF0YXNldFZlcnNpb246OQ==')


In [8]:
print(dir(dataset))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_keys', 'as_dataframe', 'examples', 'from_dict', 'id', 'version_id']


In [9]:
print(dataset.as_dataframe())

                                                 input  \
example_id                                               
RGF0YXNldEV4YW1wbGU6NTA=      {'input': '艾菲爾鐵塔位於哪裡？ '}   
RGF0YXNldEV4YW1wbGU6NTE=       {'input': '中國的長城有多長？ '}   
RGF0YXNldEV4YW1wbGU6NTI=     {'input': '最大的熱帶雨林是什麼？ '}   
RGF0YXNldEV4YW1wbGU6NTM=    {'input': '地球上最高的山是哪一座？ '}   
RGF0YXNldEV4YW1wbGU6NTQ=    {'input': '世界上最長的河流是什麼？ '}   
RGF0YXNldEV4YW1wbGU6NTU=       {'input': '蒙娜麗莎是誰畫的？ '}   
RGF0YXNldEV4YW1wbGU6NTY=        {'input': '人體有幾塊骨頭？ '}   
RGF0YXNldEV4YW1wbGU6NTc=  {'input': '太陽系中最大的行星是哪一顆？ '}   
RGF0YXNldEV4YW1wbGU6NTg=  {'input': '《羅密歐與茱麗葉》是誰寫的？ '}   
RGF0YXNldEV4YW1wbGU6NTk=  {'input': '第一次登月是什麼時候發生的？ '}   

                                                                     output  \
example_id                                                                    
RGF0YXNldEV4YW1wbGU6NTA=                        {'output': '艾菲爾鐵塔位於法國巴黎。 '}   
RGF0YXNldEV4YW1wbGU6NTE=       {'output': '中國的長城長約 13,171 英里（21,19

In [10]:
df = dataset.as_dataframe()
print(df.columns)

Index(['input', 'output', 'metadata'], dtype='object')


In [11]:
df["input"] = df["input"].apply(lambda x: x['input'])
df["output"] = df["output"].apply(lambda x: x['output'])


In [9]:
MY_CUSTOM_TEMPLATE = '''
    You are evaluating whether the output mentions any person's name, such as common first names, full names, or references to individuals.
    [BEGIN DATA]
    ************
    [Question]: {input}
    ************
    [Response]: {output}
    [END DATA]


    Please focus on the tone of the response.
    Your answer must be single word, either "yes" or "no"
    '''

In [12]:
import openai
import os
from phoenix.evals import llm_classify, OpenAIModel

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

res = client.chat.completions.create(
model="o1",
temperature=0.5,
messages=[
{"role": "system", "content": "RAG系統的output評估器"},
{"role": "user", "content": "RAG系統output:《蒙娜麗莎》是義大利文藝復興時期藝術家李奧納多·達文西的作品，請告訴我這個回應中有無出現人名，你只能回答有或沒有"},
]
)
print(res.choices[0].message.content)


NotFoundError: <!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

In [13]:
import openai
import os
from phoenix.evals import llm_classify, OpenAIModel

rails = ['yes', 'no']

model = OpenAIModel(
    model="o1",
    temperature=0.5,
    api_key=os.getenv("OPENAI_API_KEY"),     # 你的API Key
    base_url=os.getenv("OPENAI_API_BASE")    # 你的 Base URL
)

# 在外面跑 llm_classify（批次對整個df做分類）
positive_eval = llm_classify(
    data=df,
    template=MY_CUSTOM_TEMPLATE,
    model=model,
    rails=rails,
    concurrency=5
)


🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


llm_classify |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

In [14]:
print(positive_eval.label)

example_id
RGF0YXNldEV4YW1wbGU6NTA=     no
RGF0YXNldEV4YW1wbGU6NTE=     no
RGF0YXNldEV4YW1wbGU6NTI=     no
RGF0YXNldEV4YW1wbGU6NTM=     no
RGF0YXNldEV4YW1wbGU6NTQ=     no
RGF0YXNldEV4YW1wbGU6NTU=    yes
RGF0YXNldEV4YW1wbGU6NTY=     no
RGF0YXNldEV4YW1wbGU6NTc=     no
RGF0YXNldEV4YW1wbGU6NTg=    yes
RGF0YXNldEV4YW1wbGU6NTk=     no
Name: label, dtype: object


In [15]:
labels_mapping = positive_eval["label"].to_dict()
print(labels_mapping)

{'RGF0YXNldEV4YW1wbGU6NTA=': 'no', 'RGF0YXNldEV4YW1wbGU6NTE=': 'no', 'RGF0YXNldEV4YW1wbGU6NTI=': 'no', 'RGF0YXNldEV4YW1wbGU6NTM=': 'no', 'RGF0YXNldEV4YW1wbGU6NTQ=': 'no', 'RGF0YXNldEV4YW1wbGU6NTU=': 'yes', 'RGF0YXNldEV4YW1wbGU6NTY=': 'no', 'RGF0YXNldEV4YW1wbGU6NTc=': 'no', 'RGF0YXNldEV4YW1wbGU6NTg=': 'yes', 'RGF0YXNldEV4YW1wbGU6NTk=': 'no'}


In [16]:
from phoenix.experiments.types import Example

def my_task(example: Example) -> str:
    return example
    # return {"a": 1, "b": 2}

In [17]:
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

# Define an evaluator.
def name_appear(input, output) -> str:      
    output_message = output["output"]["output"]
    print(output_message)    
    
    MY_CUSTOM_TEMPLATE = f'''
You are evaluating whether the output mentions any person's name, such as common first names, full names, or references to individuals.
[BEGIN DATA]
[Response]: {output_message}
[END DATA]


Please focus on the tone of the response.
Your answer must be single word, either "yes" or "no"
'''
    
    # print(MY_CUSTOM_TEMPLATE)
    
    res = client.chat.completions.create(
        model="o1",
        temperature=0,
        messages=[
            {"role": "user", "content": MY_CUSTOM_TEMPLATE},
        ]
    )
    
    content = res.choices[0].message.content
    print(content)
    content = content.strip().lower() 
    
    return content 
    
    
def hallucination_test(input, output) -> str:
    print(input)
    # print(output)    
    input_message = input["input"]
    print(input_message)
    
    output_message = output["output"]["output"]
    print(output_message)    
    
    MY_CUSTOM_TEMPLATE = f'''

[BEGIN DATA]
[Question]: {input_message}
[Response]: {output_message}
[END DATA]

請判斷[Response]回答[Question]的內容中，是否有與事實不符的地方。
如果有，則回答 "幻覺"；沒有幻覺的話，則回答 "事實"。

不要其他說明，必須回答單詞即可。
'''
    
    # print(MY_CUSTOM_TEMPLATE)
    
    res = client.chat.completions.create(
        model="o1",
        temperature=0,
        messages=[
            {"role": "user", "content": MY_CUSTOM_TEMPLATE},
        ]
    )
    
    content = res.choices[0].message.content
    print(content)
    content = content.strip().lower() 
    
    return content
    # return "yes" if res.choices[0].message.content == "yes" else "no" 
    
    
# Store the evaluators for later use
evaluators = [name_appear, hallucination_test]

In [18]:
# Run an experiment
from phoenix.experiments import run_experiment

experiment = run_experiment(dataset, my_task, evaluators=evaluators)

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(
🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


🧪 Experiment started.
📺 View dataset experiments: http://192.168.1.122:6006/datasets/RGF0YXNldDo3/experiments
🔗 View this experiment: http://192.168.1.122:6006/datasets/RGF0YXNldDo3/compare?experimentId=RXhwZXJpbWVudDozNQ==


running tasks |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(
🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/20 (0.0%) | ⏳ 00:00<? | ?it/s

艾菲爾鐵塔位於法國巴黎。 
no
{'input': '艾菲爾鐵塔位於哪裡？ '}
艾菲爾鐵塔位於哪裡？ 
艾菲爾鐵塔位於法國巴黎。 


/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


事實
中國的長城長約 13,171 英里（21,196 公里）。 
no
{'input': '中國的長城有多長？ '}
中國的長城有多長？ 
中國的長城長約 13,171 英里（21,196 公里）。 
事實
亞馬遜雨林是世界上最大的熱帶雨林。它是世界上動植物種類最多的家園。 
no
{'input': '最大的熱帶雨林是什麼？ '}
最大的熱帶雨林是什麼？ 
亞馬遜雨林是世界上最大的熱帶雨林。它是世界上動植物種類最多的家園。 
事實
珠穆朗瑪峰海拔 29,029 英尺（8,848 公尺），是地球上最高的山峰。 
no
{'input': '地球上最高的山是哪一座？ '}
地球上最高的山是哪一座？ 
珠穆朗瑪峰海拔 29,029 英尺（8,848 公尺），是地球上最高的山峰。 
事實
尼羅河全長 6,650 公里（4,132 英里），是世界上最長的河流。 
no
{'input': '世界上最長的河流是什麼？ '}
世界上最長的河流是什麼？ 
尼羅河全長 6,650 公里（4,132 英里），是世界上最長的河流。 
幻覺
《蒙娜麗莎》是義大利文藝復興時期藝術家李奧納多·達文西的作品。 
yes
{'input': '蒙娜麗莎是誰畫的？ '}
蒙娜麗莎是誰畫的？ 
《蒙娜麗莎》是義大利文藝復興時期藝術家李奧納多·達文西的作品。 
事實
成年人的身體通常有 256 塊骨頭。 
no
{'input': '人體有幾塊骨頭？ '}
人體有幾塊骨頭？ 
成年人的身體通常有 256 塊骨頭。 
幻覺
木星是我們太陽系中最大的行星。 
no
{'input': '太陽系中最大的行星是哪一顆？ '}
太陽系中最大的行星是哪一顆？ 
木星是我們太陽系中最大的行星。 
事實
戲劇《羅密歐與茱麗葉》是威廉·莎士比亞創作的。 
yes
{'input': '《羅密歐與茱麗葉》是誰寫的？ '}
《羅密歐與茱麗葉》是誰寫的？ 
戲劇《羅密歐與茱麗葉》是威廉·莎士比亞創作的。 
事實
第一次登月發生在 1969 年 7 月 20 日。 
no
{'input': '第一次登月是什麼時候發生的？ '}
第一次登月是什麼時候發生的？ 
第一次登月發生在 1969 年 7 月 20 日。 
事實

🔗 View this experiment: http://192.168.1.122:6006/d

In [5]:
df["metadata"] = df["metadata"].apply(lambda x: x['reference'])


In [6]:
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

def metadata_test(input, output, metadata) -> str:
    input_message = input["input"]
    output_message = output["output"]
    reference_message = metadata["reference"]
    print(input_message)
    print(output_message)
    print(reference_message)
    
    MY_CUSTOM_TEMPLATE = f'''

[BEGIN DATA]
[Question]: {input_message}
[Response]: {output_message}
[Reference]: {reference_message}
[END DATA]

請根據[Reference]的內容，判斷[Response]對於[Question]的回答中，是否有與事實不符的地方。
如果有，則回答 "幻覺"；沒有幻覺的話，則回答 "事實"。

不要其他說明，必須回答單詞即可。
'''
    
    res = client.chat.completions.create(
        model="o1",
        temperature=0,
        messages=[
            {"role": "user", "content": MY_CUSTOM_TEMPLATE},
        ]
    )
    
    content = res.choices[0].message.content
    print(content)
    content = content.strip().lower() 
    
    return content
    # return "yes" if res.choices[0].message.content == "yes" else "no" 
    
    
# Store the evaluators for later use
evaluators = [metadata_test]

NameError: name 'openai' is not defined

In [23]:
# Run an experiment
from phoenix.experiments import run_experiment

experiment = run_experiment(dataset, my_task, evaluators=evaluators)

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(
🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


🧪 Experiment started.
📺 View dataset experiments: http://192.168.1.122:6006/datasets/RGF0YXNldDo3/experiments
🔗 View this experiment: http://192.168.1.122:6006/datasets/RGF0YXNldDo3/compare?experimentId=RXhwZXJpbWVudDozNg==


running tasks |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(
🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

艾菲爾鐵塔位於哪裡？ 
{'output': '艾菲爾鐵塔位於法國巴黎。 '}
艾菲爾鐵塔位於法國巴黎。它建於 1889 年，作為 1889 年世界博覽會的入口拱門。 
事實
中國的長城有多長？ 
{'output': '中國的長城長約 13,171 英里（21,196 公里）。 '}
中國的長城長達 13,000 多英里。它是中國各個朝代歷經多個世紀而建造的，旨在抵禦遊牧民族的入侵。 


/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


事實
最大的熱帶雨林是什麼？ 
{'output': '亞馬遜雨林是世界上最大的熱帶雨林。它是世界上動植物種類最多的家園。 '}
亞馬遜雨林是世界上最大的熱帶雨林。它覆蓋了巴西西北部的大部分地區，並延伸到哥倫比亞、秘魯和其他南美國家。 
事實
地球上最高的山是哪一座？ 
{'output': '珠穆朗瑪峰海拔 29,029 英尺（8,848 公尺），是地球上最高的山峰。 '}
珠穆朗瑪峰是地球上最高的山峰。它位於喜馬拉雅山馬哈蘭古爾喜馬拉雅山脈，橫跨尼泊爾和西藏邊界。 
事實
世界上最長的河流是什麼？ 
{'output': '尼羅河全長 6,650 公里（4,132 英里），是世界上最長的河流。 '}
尼羅河是世界上最長的河流。它向北流經非洲東北部，從布隆迪最遠的源頭流入地中海，全長約 6,650 公里（4,132 英里）。 
事實
蒙娜麗莎是誰畫的？ 
{'output': '《蒙娜麗莎》是義大利文藝復興時期藝術家李奧納多·達文西的作品。 '}
《蒙娜麗莎》是李奧納多達文西畫的。它被認為是義大利文藝復興時期的典型傑作，並被描述為「世界上最著名、訪問量最大、被書寫最多、被傳頌最多、被模仿最多的藝術作品」。 
事實
人體有幾塊骨頭？ 
{'output': '成年人的身體通常有 256 塊骨頭。 '}
人體有206塊骨頭。這些骨骼提供結構、保護器官、固定肌肉並儲存鈣。 
幻覺
太陽系中最大的行星是哪一顆？ 
{'output': '木星是我們太陽系中最大的行星。 '}
木星是我們太陽系中最大的行星。它是一顆氣態巨行星，質量是太陽系中所有其他行星質量總和的兩倍半以上。 
事實
《羅密歐與茱麗葉》是誰寫的？ 
{'output': '戲劇《羅密歐與茱麗葉》是威廉·莎士比亞創作的。 '}
威廉·莎士比亞寫了《羅密歐與茱麗葉》。這是一場關於兩個不幸的年輕戀人的悲劇，他們的死亡最終使他們不和的家庭和解。 
事實
第一次登月是什麼時候發生的？ 
{'output': '第一次登月發生在 1969 年 7 月 20 日。 '}
第一次登月發生在 1969 年。 1969 年 7 月 20 日，美國太空人尼爾阿姆斯壯和埃德溫「巴茲」奧爾德林作為阿波羅 11 號任務的一部分，成為第一批登陸月球的人類。 
事實

🔗 View this experiment: http://192.1

In [24]:
import pandas as pd

# 執行實驗
experiment = run_experiment(dataset, my_task, evaluators=evaluators)

# 從 dataset 與 experiment 中提取評估結果
records = []

for row in dataset.iter_examples():
    input_text = row["input"]["input"]
    output_text = row["output"]["output"]
    reference_text = row["metadata"]["reference"]
    
    # 從 Phoenix 實驗結果中抓取該 example 的判斷
    example_id = row["example_id"]
    result = experiment.results[example_id]["metadata_test"]

    records.append({
        "input": input_text,
        "output": output_text,
        "reference": reference_text,
        "判斷結果": result
    })

# 存成 .ods（需要安裝 odfpy：pip install odfpy）
df = pd.DataFrame(records)
df.to_excel("幻覺判斷結果.ods", engine="odf", index=False)


/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(
🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


🧪 Experiment started.
📺 View dataset experiments: http://192.168.1.122:6006/datasets/RGF0YXNldDo3/experiments
🔗 View this experiment: http://192.168.1.122:6006/datasets/RGF0YXNldDo3/compare?experimentId=RXhwZXJpbWVudDozNw==


running tasks |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(
🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

艾菲爾鐵塔位於哪裡？ 
{'output': '艾菲爾鐵塔位於法國巴黎。 '}
艾菲爾鐵塔位於法國巴黎。它建於 1889 年，作為 1889 年世界博覽會的入口拱門。 
事實
中國的長城有多長？ 
{'output': '中國的長城長約 13,171 英里（21,196 公里）。 '}
中國的長城長達 13,000 多英里。它是中國各個朝代歷經多個世紀而建造的，旨在抵禦遊牧民族的入侵。 


/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


事實
最大的熱帶雨林是什麼？ 
{'output': '亞馬遜雨林是世界上最大的熱帶雨林。它是世界上動植物種類最多的家園。 '}
亞馬遜雨林是世界上最大的熱帶雨林。它覆蓋了巴西西北部的大部分地區，並延伸到哥倫比亞、秘魯和其他南美國家。 
事實
地球上最高的山是哪一座？ 
{'output': '珠穆朗瑪峰海拔 29,029 英尺（8,848 公尺），是地球上最高的山峰。 '}
珠穆朗瑪峰是地球上最高的山峰。它位於喜馬拉雅山馬哈蘭古爾喜馬拉雅山脈，橫跨尼泊爾和西藏邊界。 
事實
世界上最長的河流是什麼？ 
{'output': '尼羅河全長 6,650 公里（4,132 英里），是世界上最長的河流。 '}
尼羅河是世界上最長的河流。它向北流經非洲東北部，從布隆迪最遠的源頭流入地中海，全長約 6,650 公里（4,132 英里）。 
事實
蒙娜麗莎是誰畫的？ 
{'output': '《蒙娜麗莎》是義大利文藝復興時期藝術家李奧納多·達文西的作品。 '}
《蒙娜麗莎》是李奧納多達文西畫的。它被認為是義大利文藝復興時期的典型傑作，並被描述為「世界上最著名、訪問量最大、被書寫最多、被傳頌最多、被模仿最多的藝術作品」。 
事實
人體有幾塊骨頭？ 
{'output': '成年人的身體通常有 256 塊骨頭。 '}
人體有206塊骨頭。這些骨骼提供結構、保護器官、固定肌肉並儲存鈣。 
幻覺
太陽系中最大的行星是哪一顆？ 
{'output': '木星是我們太陽系中最大的行星。 '}
木星是我們太陽系中最大的行星。它是一顆氣態巨行星，質量是太陽系中所有其他行星質量總和的兩倍半以上。 
事實
《羅密歐與茱麗葉》是誰寫的？ 
{'output': '戲劇《羅密歐與茱麗葉》是威廉·莎士比亞創作的。 '}
威廉·莎士比亞寫了《羅密歐與茱麗葉》。這是一場關於兩個不幸的年輕戀人的悲劇，他們的死亡最終使他們不和的家庭和解。 
事實
第一次登月是什麼時候發生的？ 
{'output': '第一次登月發生在 1969 年 7 月 20 日。 '}
第一次登月發生在 1969 年。 1969 年 7 月 20 日，美國太空人尼爾阿姆斯壯和埃德溫「巴茲」奧爾德林作為阿波羅 11 號任務的一部分，成為第一批登陸月球的人類。 
事實

🔗 View this experiment: http://192.1

AttributeError: 'Dataset' object has no attribute 'iter_examples'